In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from numpy import asarray
from numpy import sqrt

from pandas import concat
from pandas import DataFrame


from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras import Sequential

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM

In [ ]:
#function to get well data given the well code
def get_df(well_code):
    return df_dict[well_code]

#function to get test & train data given the well code
def get_test_and_train_data(well_code):

    well_df = get_df(well_code)

    X = well_df.drop(columns = ["BORE_OIL_VOL", "BORE_GAS_VOL", "BORE_WAT_VOL"])
    y = well_df[["BORE_OIL_VOL", "BORE_GAS_VOL", "BORE_WAT_VOL"]]
    
    return train_test_split(X, y, test_size = 0.3, random_state=42)

def get_outputs(well_code):
    well_df = get_df(well_code)
    
    return tuple([well_df[col] for col in ["BORE_OIL_VOL", "BORE_GAS_VOL", "BORE_WAT_VOL"]])

#print (get_outputs(7405)[0])


# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    
    for i in range(len(sequence)):
        
        # find the end of this pattern
        end_ix = i + n_steps
        
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
            
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
        
    return asarray(X), asarray(y)

In [ ]:
well_code = 5599
well_df = get_df(well_code)
X_train, X_test, y_train, y_test = get_test_and_train_data(well_code)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

plot_cols = ["BORE_OIL_VOL", "BORE_GAS_VOL", "BORE_WAT_VOL"]
plot_features = y_train[plot_cols]
plot_features.index = date[:y_train.shape[0]]
_ = plot_features.plot(subplots=True)

plot_features = y_train[plot_cols][:480]
plot_features.index = date[:480]
_ = plot_features.plot(subplots=True)

plt.show()

In [ ]:
# load the dataset
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-car-sales.csv'
df = read_csv(path, header=0, index_col=0, squeeze=True)

# retrieve the values
values = df.values.astype('float32')

# specify the window size
n_steps = 5

# split into samples
X, y = split_sequence(values, n_steps)

# reshape into [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

# split into train/test
n_test = 12
X_train, X_test, y_train, y_test = X[:-n_test], X[-n_test:], y[:-n_test], y[-n_test:]
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', kernel_initializer='he_normal', input_shape=(n_steps,1)))
model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1))

# compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# fit the model
model.fit(X_train, y_train, epochs=350, batch_size=32, verbose=2, validation_data=(X_test, y_test))

In [ ]:
# evaluate the model
mse, mae = model.evaluate(X_test, y_test, verbose=0)
print('MSE: %.3f, RMSE: %.3f, MAE: %.3f' % (mse, sqrt(mse), mae))

# make a prediction
row = asarray([18024.0, 16722.0, 14385.0, 21342.0, 17180.0]).reshape((1, n_steps, 1))
yhat = model.predict(row)
print('Predicted: %.3f' % (yhat))